## calculate cosine 

In [17]:
import numpy as np
import torch
import os
import json
import pandas as pd
import multiprocessing as mp
from transformers import BertTokenizerFast
# sys.path.append('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM')
BERT_PRETRAIN_MODEL = "dmis-lab/biobert-base-cased-v1.2"
TOKENIZER = BertTokenizerFast.from_pretrained(BERT_PRETRAIN_MODEL, do_lower_case=True)


In [23]:
def read_data(readPath):
    df = pd.read_json(readPath, lines=True)
    return df

In [57]:
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def cosine_module(a, b, cosine_sum):
    norm_array1 = np.linalg.norm(a)
    norm_array2 = np.linalg.norm(b)
    
    module_similarity = 1 - (np.abs(norm_array1 - norm_array2) / (norm_array1 + norm_array2))
    
    return module_similarity * cosine_sum
 

def cosine_module_2_tensors(tensor1, tensor2, cosine_sum_m):

    norm_tensor1 = torch.norm(tensor1)
    norm_tensor2 = torch.norm(tensor2)
    
    # cosine_sim_v2 = cosine_func(tensor1 / norm_tensor1, tensor2 / norm_tensor2)
    module_similarity = 1 - (torch.abs(norm_tensor1 - norm_tensor2) / (norm_tensor1 + norm_tensor2))
    
    return module_similarity * cosine_sum_m


In [54]:
import time

def cosine_sen_content_word(sum_tensor, avg_tensor, content_word_dict):
    '''một từ với 1 content word
    trả ra consine similarity giữa sum_vector_tensor và sum_vector của content word
    '''
   
    content_tensor_sum = content_word_dict['sum_vector']
    # content_tensor_avg = torch.tensor(content_word_dict['avg_vector']).clone().detach()
    t0 = time.time()
    # cosine_sum = cosine_func(sum_tensor/torch.norm(sum_tensor), content_tensor_sum/torch.norm(content_tensor_sum)).item()
    cosine_sum = cosine_sim(sum_tensor, content_tensor_sum)
    cosine_module_sum = cosine_module(sum_tensor, content_tensor_sum, cosine_sum)
    print("time cu", time.time() - t0)
    print("cosine cu", cosine_sum, cosine_module_sum)
    print("=======")
    
    sum_vector_tensor = torch.tensor(sum_tensor).clone().detach()
    # avg_vector_tensor = torch.tensor(avg_tensor).clone().detach()
    content_tensor_sum_tensor = torch.tensor(content_tensor_sum).clone().detach()
    t1 = time.time()
    cosine_func = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
    cosine_sum_m = cosine_func(sum_vector_tensor/torch.norm(sum_vector_tensor), content_tensor_sum_tensor/torch.norm(content_tensor_sum_tensor)).item()
    cosine_module_sum_m = cosine_module_2_tensors(sum_vector_tensor, content_tensor_sum_tensor, cosine_sum_m)
    print("time moi", time.time() - t1)
    print("cosine moi", cosine_sum_m, cosine_module_sum_m)
    # cosine_avg = cosine_func(avg_tensor/torch.norm(avg_tensor), content_tensor_avg/torch.norm(content_tensor_avg)).item()
    # cosine_module_avg = cosine_module_2_tensors(avg_tensor, content_tensor_avg, cosine_func)
    
    return cosine_sum, cosine_module_sum, content_word_dict['word'] # , cosine_avg, cosine_module_avg

In [46]:

def cosine_sim_sen_list_word(sum_tensor, avg_tensor, list_dict_content_word):
    '''tinh cosine trong 1 cau voi tat ca cac content word. trả ra từ có cosine gần -1 và từ có cosine gần 0'''
    list_result = list(map(lambda x: cosine_sen_content_word(sum_tensor, avg_tensor, x), list_dict_content_word))
    
    sorted_cos_sum = sorted(list_result, key = lambda x: x[0]) # tang dan
   
    sorted_cos_module_sum = sorted(list_result, key = lambda x: x[1]) # tang dan
    
    # sorted_cos_avg = sorted(list_result, key = lambda x: x[3]) # tang dan
    
    # sorted_cos_module_avg = sorted(list_result, key = lambda x: x[4]) # tang dan
    
    # cosine gần -1
    cos_neg_sum, _, replace_word_neg_cos_sum = sorted_cos_sum[0]
    _, cos_module_neg_sum, replace_word_neg_cos_module_sum = sorted_cos_module_sum[0]
    # _, _, replace_word_neg_cos_avg, cos_avg, _ = sorted_cos_avg[-1]
    # _, _, replace_word_neg_cos_module_avg, _, cos_module_avg = sorted_cos_module_avg[-1]
    
    # cosine gần 0
    sorted_cos_sum.sort(key=lambda x: abs(x[0]))
    cos_pos_sum, _, replace_word_0_cos_sum = sorted_cos_sum[0]
   
    sorted_cos_module_sum.sort(key=lambda x: abs(x[1]))
    _, cos_module_pos_sum, replace_word_0_cos_module_sum = sorted_cos_module_sum[0]
   
    
    # abs_list_cos_avg = list(map(lambda x: abs(x[3] - 0), sorted_cos_avg))
    # min_abs_cos_avg = min(abs_list_cos_avg)
    # _, _, replace_word_0_cos_avg, cos_pos_avg, _ = sorted_cos_avg[abs_list_cos_avg.index(min_abs_cos_avg)]
    
    # abs_list_cos_module_avg = list(map(lambda x: abs(x[4] - 0), sorted_cos_module_avg))
    # min_abs_cos_module_avg = min(abs_list_cos_module_avg)
    # _, _, replace_word_0_cos_module_avg, _, cos_module_pos_avg = sorted_cos_module_avg[abs_list_cos_module_avg.index(min_abs_cos_module_avg)]
    
    return replace_word_neg_cos_sum, replace_word_neg_cos_module_sum, replace_word_0_cos_sum, replace_word_0_cos_module_sum #, replace_word_neg_cos_avg, replace_word_neg_cos_module_avg, replace_word_0_cos_avg, replace_word_0_cos_module_avg

In [20]:
def replace_word_in_sentence(predicate_data_chunk, chunkNumber, tempList, content_word_data):
    ''' 1 câu gốc thành 2 câu mới: cosine = -1 và cosine = 0
    '''
    name = 'new_data_{}.json'.format(str(chunkNumber))

    # cosine_func = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
    def new_word_in_sen(predicate_data):
       
        # masked_index = torch.where(torch.tensor(predicate_data['pos_tag_id']).clone().detach() != 0)
        masked_index = np.where(np.array(predicate_data['pos_tag_id']) != 0)

        predicate_np = np.array(predicate_data['origin_id'])
        
        avg_tensor = predicate_data['avg_vector']
        sum_tensor = predicate_data['sum_vector']
        
        content_word = None
        pos_tag = None
        if predicate_data['pos_tag_id'][masked_index[0][0].item()] == 1:
            content_word = content_word_data['noun']
            pos_tag = 'noun'
            
        elif predicate_data['pos_tag_id'][masked_index[0][0].item()] == 2:
            content_word = content_word_data['verb']
            pos_tag = 'verb'
        
        elif predicate_data['pos_tag_id'][masked_index[0][0].item()] == 3:
            content_word = content_word_data['adj']
            pos_tag = 'adj'
        
        elif predicate_data['pos_tag_id'][masked_index[0][0].item()] == 4:
            content_word = content_word_data['adv']
            pos_tag = 'adv'
        else: 
            KeyError('pos_tag_id not in [1, 2, 3, 4]')
        
        replace_word_neg_cos, replace_word_neg_cos_module, replace_word_0_cos, replace_word_0_cos_module = cosine_sim_sen_list_word(sum_tensor, avg_tensor, content_word)

        # cosine = -1
        origin_word = TOKENIZER.decode(predicate_np[masked_index[0]])
        # origin_sen = TOKENIZER.decode(predicate_np, skip_special_tokens = True)
        # cos_neg_sen_sum = origin_sen.replace(origin_word, TOKENIZER.decode(replace_word_cosine_neg_sum), 1) 
        cos_neg = TOKENIZER.decode(replace_word_neg_cos)
        cos_module_neg = TOKENIZER.decode(replace_word_neg_cos_module)
        
        # cosine = 0
        cos_0 = TOKENIZER.decode(replace_word_0_cos)
        cos_module_0 = TOKENIZER.decode(replace_word_0_cos_module)
        
        feature =  {
                "origin_uid": predicate_data['origin_uid'],
                "origin_id": predicate_data['origin_id'].tolist(), 
                "masked_index": masked_index[0].tolist(),
                "masked_word": origin_word,
                "cos_neg": cos_neg, 
                "cos_0": cos_0, 
                "cos_module_neg": cos_module_neg,
                "cos_module_0": cos_module_0,
                "pos_tag": pos_tag}
        return feature         
                
    # list_feature = map(lambda x: new_word_in_sen(x), tqdm(predicate_data_chunk))
    list_feature =  new_word_in_sen(predicate_data_chunk[0])          
    with open(name, 'w') as wf:
        for feature in list_feature:
            wf.write('{}\n'.format(json.dumps(feature))) 
        tempList.append(name)

In [14]:
def create_new_data(predicate_file, dict_content_word, wriDir):
    '''
    return 4 data cho moi file:
        sum_vector va cosine = -1: uid, origin_input_id, sum_neg_input_id, pos_tag_id
        sum_vector va cosine = 0: uid, origin_input_id, sum_0_input_id, pos_tag_id
        avg_vector va cosine = -1: uid, origin_input_id, avg_neg_input_id, pos_tag_id
        avg_vector va cosine = 0: uid, origin_input_id, avg_0_input_id, pos_tag_id
    
    '''
    print("Preprocessing file... ", predicate_file)
    predicates_data = read_data(predicate_file)[:10]
    # predicates_data = [item for sublist in predicates_data for item in sublist][:10]
    
    # MULTI PROCESSING
    man = mp.Manager()

    # shared list to store all temp files written by processes
    tempFilesList = man.list()
    
    # numProcess = mp.cpu_count() - 1
    numProcess = 1
    
    
    chunkSize = int(len(predicates_data) / (numProcess))
    print('Data Size: ', len(predicates_data))
    print('number of threads: ', numProcess)
  
    processes = []
    for i in range(numProcess):
        dataChunk = predicates_data[chunkSize*i : chunkSize*(i+1)]

        p = mp.Process(target = replace_word_in_sentence, args = (dataChunk, i, tempFilesList, dict_content_word))
        
        p.start()
        processes.append(p)
        
    for pr in processes:
        pr.join()
    
    wrtPath = wriDir + '{}'.format(predicate_file.split('/')[-1].replace('mlm_', ''))
    
    # combining the files written by multiple processes into a single final file
    with open(wrtPath, 'w') as f:
        for file in tempFilesList:
            with open(file, 'r') as r:
                for line in r:
                    sample =  json.loads(line)
                    f.write('{}\n'.format(json.dumps(sample)))
            os.remove(file)
        
    print("Done file", predicate_file)   


## khong chay nua


In [6]:

file_paths = {
    "noun": "./data_mlm/process_folder/list_content_word_v2/NOUN.json",
    "verb": "./data_mlm/process_folder/list_content_word_v2/VERB.json",
    "adj": "./data_mlm/process_folder/list_content_word_v2/ADJ.json",
    "adv": "./data_mlm/process_folder/list_content_word_v2/ADV.json"
}


dict_content_word = {key: read_data(file_path) for key, file_path in file_paths.items()}


In [8]:
len(dict_content_word['noun']), len(dict_content_word['verb']), len(dict_content_word['adj']), len(dict_content_word['adv'])

(42549, 22676, 13854, 4537)

## chay tiep

In [14]:
from mlm_utils.transform_func import get_files
# wriDir = './pertured_data/'
wriDir = './data_mlm/process_folder'
dataDir = './data_mlm/process_folder/word_present_each_file/'
files = get_files(dataDir)

# for file in files:
    
# create_new_data(dataDir + files[0], dict_content_word, wriDir)

/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-05-14 15:11:12.600124: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 15:11:13.237953: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 15:11:15.725031: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-14 15:11:18.315892: E tensorflow/compiler/xla/stream_executor/cuda/cu

In [55]:
def new_word_in_sen(predicate_data, content_word_data):
       
    # masked_index = torch.where(torch.tensor(predicate_data['pos_tag_id']).clone().detach() != 0)
    masked_index = np.where(np.array(predicate_data['pos_tag_id']) != 0)
    print("masked index", masked_index)
    predicate_np = np.array(predicate_data['origin_id'])
    
    avg_tensor = predicate_data['avg_vector']
    sum_tensor = predicate_data['sum_vector']
    
    content_word = None
    pos_tag = None
    if predicate_data['pos_tag_id'][masked_index[0][0].item()] == 1:
        content_word = content_word_data['noun']
        pos_tag = 'noun'
        
    elif predicate_data['pos_tag_id'][masked_index[0][0].item()] == 2:
        content_word = content_word_data['verb']
        pos_tag = 'verb'
    
    elif predicate_data['pos_tag_id'][masked_index[0][0].item()] == 3:
        content_word = content_word_data['adj']
        pos_tag = 'adj'
    
    elif predicate_data['pos_tag_id'][masked_index[0][0].item()] == 4:
        content_word = content_word_data['adv']
        pos_tag = 'adv'
    else: 
        KeyError('pos_tag_id not in [1, 2, 3, 4]')
    
    replace_word_neg_cos, replace_word_neg_cos_module, replace_word_0_cos, replace_word_0_cos_module = cosine_sim_sen_list_word(sum_tensor, avg_tensor, content_word)

    # cosine = -1
    origin_word = TOKENIZER.decode(predicate_np[masked_index[0]])
    # origin_sen = TOKENIZER.decode(predicate_np, skip_special_tokens = True)
    # cos_neg_sen_sum = origin_sen.replace(origin_word, TOKENIZER.decode(replace_word_cosine_neg_sum), 1) 
    cos_neg = TOKENIZER.decode(replace_word_neg_cos)
    cos_module_neg = TOKENIZER.decode(replace_word_neg_cos_module)
    
    # cosine = 0
    cos_0 = TOKENIZER.decode(replace_word_0_cos)
    cos_module_0 = TOKENIZER.decode(replace_word_0_cos_module)
    
    feature =  {
            "origin_uid": predicate_data['origin_uid'],
            "origin_id": predicate_data['origin_id'], 
            "masked_index": masked_index[0].tolist(),
            "masked_word": origin_word,
            "cos_neg": cos_neg, 
            "cos_0": cos_0, 
            "cos_module_neg": cos_module_neg,
            "cos_module_0": cos_module_0,
            "pos_tag": pos_tag}
    return feature    

In [61]:
predicates_data = read_data(dataDir + files[0])[:10]
# convert to dataframe
import pandas as pd
df = pd.DataFrame(predicates_data)
df.head()

,origin_uid,origin_id,pos_tag_id,word,sum_vector,avg_vector
0,0,"[101, 170, 176, 118, 1106, 118, 170, 6468, 112...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",[6468],"[-0.582668125629425, 0.7431492805480957, -0.67...","[-0.582668125629425, 0.7431492805480957, -0.67..."
1,0,"[101, 170, 176, 118, 1106, 118, 170, 6468, 112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...",[1148],"[-0.5867326855659485, 1.362990140914917, -0.44...","[-0.5867326855659485, 1.362990140914917, -0.44..."
2,0,"[101, 170, 176, 118, 1106, 118, 170, 6468, 112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[183, 21977, 26918, 23767]","[-2.1963400840759277, 4.488198757171631, -4.05...","[-0.5490850210189819, 1.1220496892929077, -1.0..."
3,0,"[101, 170, 176, 118, 1106, 118, 170, 6468, 112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[27553, 1179]","[-0.9821252822875977, 1.8156334161758423, -2.4...","[-0.49106264114379883, 0.9078167080879211, -1...."
4,0,"[101, 170, 176, 118, 1106, 118, 170, 6468, 112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[5351],"[-0.13667286932468414, 0.5582464933395386, -0....","[-0.13667286932468414, 0.5582464933395386, -0...."


In [20]:
df_noun = read_data("/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/data_mlm/process_folder/word_present_each_file/mlm_abolish_full.json")

df_noun.head()

,origin_uid,origin_id,pos_tag_id,word,sum_vector,avg_vector
0,0,"[101, 170, 176, 118, 1106, 118, 170, 6468, 112...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",[6468],"[-0.582668125629425, 0.7431492805480957, -0.67...","[-0.582668125629425, 0.7431492805480957, -0.67..."
1,0,"[101, 170, 176, 118, 1106, 118, 170, 6468, 112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...",[1148],"[-0.5867326855659485, 1.362990140914917, -0.44...","[-0.5867326855659485, 1.362990140914917, -0.44..."
2,0,"[101, 170, 176, 118, 1106, 118, 170, 6468, 112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[183, 21977, 26918, 23767]","[-2.1963400840759277, 4.488198757171631, -4.05...","[-0.5490850210189819, 1.1220496892929077, -1.0..."
3,0,"[101, 170, 176, 118, 1106, 118, 170, 6468, 112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[27553, 1179]","[-0.9821252822875977, 1.8156334161758423, -2.4...","[-0.49106264114379883, 0.9078167080879211, -1...."
4,0,"[101, 170, 176, 118, 1106, 118, 170, 6468, 112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[5351],"[-0.13667286932468414, 0.5582464933395386, -0....","[-0.13667286932468414, 0.5582464933395386, -0...."
